In [1]:
"""
all of the packages needed in this notebook and their versions.
"""

!mamba install tensorflow-gpu==2.11.0 -y -q
!mamba install pytorch-cuda=11.6 -c pytorch -c conda-forge -c nvidia -y -q
!mamba install -c conda-forge pyts==0.12.0 -y -q
!pip install torch==1.13 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 -q
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.1+cu116.html -q

!pip install ts2vg==1.2.1 -q
!pip install pytorch_lightning==1.9.1 -q
!pip install torchsummary==1.5.1 -q
!pip install dvclive==2.0.2 -q

!pip install -e GraphXAI/ #==0.1
!pip install pyedflib==0.1.33 -q
!pip install mne==1.4.2 -q
!pip install ipdb==0.13.13 -q
!pip install pgmpy==0.1.23 -q

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... By downloading and using the CUDA Toolkit conda packages, you accept the terms and conditions of the CUDA End User License Agreement (EULA): https://docs.nvidia.com/cuda/eula/index.html

By downloading and using the cuDNN conda packages, you accept the terms and conditions of the NVIDIA cuDNN EULA -
  https://docs.nvidia.com/deeplearning/cudnn/sla/index.html

done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [13]:
pip show mne

Name: mne
Version: 1.4.2
Summary: MNE-Python project for MEG and EEG data analysis.
Home-page: https://mne.tools/dev/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /opt/conda/lib/python3.10/site-packages
Requires: decorator, jinja2, matplotlib, numpy, packaging, pooch, scipy, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import torch
import warnings

from pyts.image import MarkovTransitionField

from ts2vg import NaturalVG
from ts2vg import HorizontalVG

from torch_geometric.data import Data

def get_Rutgers():
    """
    Gets X, labels Y for graph clasification, and mask of Y for every sample for node classification, of a chosen Rutgers time series

    Affected by:
        Config["graph"]["len_type"]
        Config["graph"]["data_path"]
        Config["graph"]["folder_name"]
        Config["graph"]["properties_name"]
        Config["graph"]["mask_name"]
        
    Returns:
       _X: a 2D array containing the time steps of the rutgers dataset
       mask: a 2D array containing the mask for node classification for the rutgers dataset
       Y: a 1D array containing the mask for graph classification for the rutgers dataset
    """
    # for the path containing time series, all with the same number of samples
    if Config["graph"]["len_type"] == "un/cut":
        df = pd.read_csv(Config["graph"]["data_path"] + Config["graph"]["folder_name"])  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = df.loc[:,df.columns[:length_rss]].to_numpy()
        Y = df[length_rss+1].to_numpy(dtype=np.uint8)
        mask = df.loc[:,df.columns[length_rss+2:]].to_numpy()
        
    # for the path containing time series, with diferent number of samples
    elif Config["graph"]["len_type"] == "random":
        dataset_rss = np.load(Config["graph"]["data_path"] + Config["graph"]["folder_name"], allow_pickle=True)['arr_0']
        dataset_properties = np.load(Config["graph"]["data_path"] + Config["graph"]["properties_name"], allow_pickle=True)['arr_0']
        dataset_mask = np.load(Config["graph"]["data_path"] + Config["graph"]["mask_name"], allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            dataset_properties[i,1] = int(dataset_properties[i,1])
        
        X = dataset_rss 
        mask = dataset_mask 
        Y = dataset_properties[:,2] 
        
    return X, mask, Y

def get_matrix(X_current):
    """
    This function gets the adjacency matrix through either visibility, MTF, or dual VG graph
    
    Affected:
        Config["graph"]["type"]
        Config["graph"]["VG"]["edge_type"]
        Config["graph"]["VG"]["edge_dir"]
        Config["graph"]["MTF"]["num_bins"]
        
    Args:
        X_current: a 1D array usually containing time series values
    
    Returns:
        adj_mat: a list of adjacency matrices
    """
    adj_mat = []
    
    if Config["graph"]["type"] in ("VG", "Dual_VG"):
        VGConfig = Config["graph"]["VG"]
        
        if VGConfig["edge_type"] == "natural":
            g = NaturalVG(weighted=VGConfig["distance"])
        elif VGConfig["edge_type"] == "horizontal":
            g = HorizontalVG(weighted=VGConfig["distance"])

        g.build(X_current)

        adj_mat_vis = np.zeros([len(X_current), len(X_current)], dtype='float')
        for x, y, q in g.edges:
            adj_mat_vis[x, y] = q
            if VGConfig["edge_dir"] == "undirected":
                adj_mat_vis[y, x] = q
        
        adj_mat.append(adj_mat_vis)
        
    elif Config["graph"]["type"] == "MTF":
        n_bins = Config["graph"]["MTF"]["num_bins"]
        if n_bins == "auto":
            n_bins = len(X_current)
        warnings.filterwarnings("ignore")
        MTF = MarkovTransitionField(n_bins=n_bins)
        X_gaf_MTF_temp = MTF.fit_transform(X_current.reshape(1, -1))
        adj_mat.append(X_gaf_MTF_temp[0])
    
    return adj_mat
    
def adjToEdgidx(adj_mat):
    """
    This function creates edge indexes and weights for a given matrix
    
    Args:
        adj_mat: a 2D array

    Returns:
        edge_index: a 2D torch array that indicates the connected values
        edge_weight: a 1D array of weights that represent the absolute distance between connected nodes or values in the time series
    """
    edge_index = torch.from_numpy(adj_mat[0]).nonzero().t().contiguous()
    row, col = edge_index
    edge_weight = adj_mat[0][row, col]
    return edge_index, edge_weight

def adjToEdgidx_Dual_VG(X_current):
    """
    Creates a dual visibility graph by first creating a directed VG from one side and then flipping and running the get_matrix function again.
    By doing this, we join these two graphs and obtain a dual VG.

    Args:
        X_current: 1D array usually containing time series values

    Returns:
        edge_index: 2D torch array that defines the connected values
        edge_weight: 2D array of weights that represent the absolute distance between every node or value in the time series
    """
    pos_adj_mat_vis = get_matrix(X_current)[0]
    neg_adj_mat_vis = get_matrix(-X_current)[0]
    edge_index = torch.from_numpy(pos_adj_mat_vis + neg_adj_mat_vis).nonzero().t().contiguous()

    # Join two edge_weight arrays
    row, col = edge_index
    edge_weight = np.zeros([len(row), 2], dtype='float')
    edge_weight[:, 0] = pos_adj_mat_vis[row, col]
    edge_weight[:, 1] = neg_adj_mat_vis[row, col]

    return edge_index, edge_weight
    
def create_graph(output, X, mask, Y):
    """
    Creates a graph in the torch geometric Data format, containing the node values x, mask values for training, testing, and validation, edge indexes, and edge attributes.

    Affected by:
        Config["graph"]["type"]
        Config["graph"]["classif"]
    
    Args:
        output: Dataset of multiple graphs (optional). New graph will be appended to this dataset.
        X: Node values (integer).
        mask: 1D array representing the mask.

    Returns:
        output: Updated dataset of multiple or singular graph.
    """
    if Config["graph"]["type"] in ("VG", "MTF"):
        edge_index, edge_weight = adjToEdgidx(get_matrix(X))
    elif Config["graph"]["type"] == "Dual_VG":
        edge_index, edge_weight = adjToEdgidx_Dual_VG(X)
    
    
    
    x = torch.unsqueeze(torch.tensor(X, dtype=torch.double), 1).clone().detach()
    edge_index = edge_index.clone().detach().to(torch.int64)
    edge_attr = torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double), 1).clone().detach()
    
    if Config["graph"]["classif"] == "graph": # for graph classification
        y = torch.tensor(Y, dtype=torch.long)
    elif Config["graph"]["classif"] == "node":# for node classification 
        y = torch.unsqueeze(torch.tensor(mask, dtype=torch.double),1)


    output.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y))
    return output

In [3]:
from sklearn.utils import class_weight
def Rutgers_graph():
    """
    Performs a pipeline of operations to process the Rutgers dataset for a given utility.

    Affected by:
        Config["graph"]

    Returns:
        output: A list containing the processed graph data.
        class_weights: Torch tensor containing the computed class weights.
    """
    # creates X, mask of lables and Y as a graph lable
    X, mask, Y = get_Rutgers()
    
    # here graphs are appended to the dataset
    dataset = []
    for i in range(len(X)):
        dataset = create_graph(dataset, X[i], mask[i],Y[i])
        
    # join all lables into a 1D array
    all_x = []
    for obj in dataset:
        all_x.append(obj.y.numpy())
    all_x = np.array(all_x).reshape(-1)
    
    # all_x = np.reshape(np.concatenate([obj.y for obj in dataset]), (-1,))
    
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                    classes=np.unique(all_x),
                                                                    y=all_x))
    if Config["graph"]["classif"] == "node":
        class_weights =torch.tensor([class_weights[1]/class_weights[0]])

    return dataset, class_weights

In [36]:
import pytorch_lightning as pl
import torch.nn.functional as F
import torch.nn as nn

from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from dvclive.lightning import DVCLiveLogger

from torch.nn import Sequential, BatchNorm1d, ReLU, Linear, CrossEntropyLoss, BCEWithLogitsLoss

from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, GATConv, global_max_pool

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
"""
This is the definition of a model arhitecture created for graph classification using GINEConv layers
""" 
class GINE(nn.Module):
    def __init__(self):
        super().__init__()
        
        if Config["graph"]["type"] in ("MTF", "VG"):
            edge_dim = 1
        elif Config["graph"]["type"] in ("dual_VG"):
            edge_dim = 2

        dim_h = 32
        # Define GINEConv and Linear layers
        self.conv1 = GINEConv(
            Sequential(Linear(dim_h, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)

        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)

        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)

        self.conv4 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)

        self.conv5 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)


        self.lin1 = Linear(dim_h*5, dim_h*5)
        self.lin2 = Linear(dim_h*5, 5)

    def forward(self, x, edge_index, forward_kwargs):
        edge_weight = forward_kwargs['edge_weight']#.to(device)
        batch = forward_kwargs['batch']#.to(device)
        x = x#.to(device)
        edge_index = edge_index#.to(device)

        # Node embeddings 
        h1 = self.conv1(x, edge_index, edge_attr=edge_weight)
        h2 = self.conv2(h1, edge_index, edge_attr=edge_weight)
        h3 = self.conv3(h2, edge_index, edge_attr=edge_weight)
        h4 = self.conv4(h3, edge_index, edge_attr=edge_weight)
        h5 = self.conv5(h4, edge_index, edge_attr=edge_weight)

        # Graph-level readout
        h1 = global_max_pool(h1, batch)
        h2 = global_max_pool(h2, batch)
        h3 = global_max_pool(h3, batch)
        h4 = global_max_pool(h4, batch)
        h5 = global_max_pool(h5, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3, h4, h5), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)

        return h

"""
This is the definition of a model arhitecture created for node classification using GATConv and Linear layers
""" 
class GAT(nn.Module):
    def __init__(self):
        super().__init__()
        # Define GATConv and Linear layers
        self.conv1 = GATConv(1, 32, heads=4)
        self.lin1 = torch.nn.Linear(1, 4 * 32)
        self.conv2 = GATConv(4 * 32, 32, heads=4)
        self.lin2 = torch.nn.Linear(4 * 32, 4 * 32)
        self.conv3 = GATConv(4 * 32, 1, heads=6,concat=False)
        self.lin3 = torch.nn.Linear(4 * 32, 1)

    def forward(self, x, edge_index, forward_kwargs):
        edge_weight = forward_kwargs['edge_weight']#.to(device)
        batch = forward_kwargs['batch']#.to(device)
        x = x#.to(device)
        edge_index = edge_index#.to(device)
        # Process input data through convolutional and linear layers
        x = F.elu(self.conv1(x, edge_index) + self.lin1(x))
        x = F.elu(self.conv2(x, edge_index) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x
    
    
"""
class NGClassifier is a Pytorch Lightning class that is used to train, validate and test a model arhitecture for graph or node classificaiton
""" 
class NGClassifier(pl.LightningModule):
    def __init__(self, class_weights, model,Config):
        super().__init__()
        self.class_weights = class_weights
        self.model = model
        self.Config = Config
        
    def configure_optimizers(self):
        """
        Configures the optimizer for training the model.
        
        Affected by:
            Config["graph"]["learning_rate"]
        
        Returns:
            optimizer: The configured optimizer.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.Config["model"]["learning_rate"], weight_decay=5e-4)
        return optimizer
    
    def acc_pred(self, out, y):
        """
        Calculates the accuracy and predicted labels that the trained model has generated.

        Affected by:
            Config["graph"]["classif"]

        Args:
            out: Output values from a trained model.
            y: True labels.

        Returns:
            accuracy: Accuracy of the model's predictions.
            pred: Predicted labels.
        """
        if self.Config["graph"]["classif"] == "graph":
            pred = out.argmax(-1)
            accuracy = (pred == y).sum() / pred.shape[0]
            
        elif self.Config["graph"]["classif"] == "node":
            preds = []
            preds.append((out > 0).float().cpu())     
            pred = torch.cat(preds, dim=0)
            accuracy = (pred == y.cpu()).sum() / pred.shape[0]
        
        return accuracy, pred
    
    def loss_function_selection(self):
        """
        Choses a loss function depending on what type of classification is happening (graph or node classification)

        Affected by:
            Config["graph"]["classif"]

        Returns:
            a loss funciton 
        """
        if self.Config["graph"]["classif"] == "graph":
            return CrossEntropyLoss(weight=self.class_weights).to(device)
        elif self.Config["graph"]["classif"] == "node":
            return BCEWithLogitsLoss(weight=self.class_weights).to(device)
            
    def training_step(self, train_batch, batch_idx):
        """
        Performs a single training step on the given batch of train data.

        Args:
            train_batch: Input data for the training step.
            batch_idx: Index of the current batch.

        Returns:
            train_loss: Loss value for the training step.
        """
        forward_kwargs={
            'edge_weight': train_batch.edge_attr,
            'batch' : train_batch.batch
        }       
        
        out = self.model(x=train_batch.x,edge_index=train_batch.edge_index,forward_kwargs=forward_kwargs)
        loss_function = self.loss_function_selection()
        train_loss = loss_function(out, train_batch.y)
        
        return train_loss
    
    def validation_step(self, val_batch, batch_idx):
        """
        Performs a single validation step on the given batch of validation data. It logs val_loss and accuracy for early stoping and saving the best trained models
        
        Args:
            val_batch: Test data for the current batch.
            batch_idx: Index of the current batch.
        """
        
        forward_kwargs={
            'edge_weight': val_batch.edge_attr,
            'batch' : val_batch.batch
        }
        out = self.model(x=val_batch.x,edge_index=val_batch.edge_index,forward_kwargs=forward_kwargs)
        loss_function = self.loss_function_selection()
        val_loss = loss_function(out, val_batch.y)
        
        accuracy, pred = self.acc_pred(out, val_batch.y)
        
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)        
    
    def test_step(self, test_batch, batch_idx):
        """
        Performs a single test step on the given batch of test data. It calculates and logs the test accuracy and returns the predicted labels and ground truth labels.

        Args:
            test_batch: Test data for the current batch.
            batch_idx: Index of the current batch.
            
        Returns:
            pred: labels predicted by the model 
            test_batch.y: labels that are true for the tested graph
        """
        # Sort the importance mask values and indices
        value, index = self.imp_mask[batch_idx].sort() #descending=True

        # Clone the input features and initialize the test mask
        x_masked = test_batch.x.clone()
        test_mask = torch.ones(1, 300 - int(self.imp_mask_index))

        edge_index = test_batch.edge_index
        edge_attr = test_batch.edge_attr

        x_masked1 = x_masked

        if self.imp_mask_index != 0:
            # Select the top k important nodes to mask
            index = index[-self.imp_mask_index:]

            # Adjust the indices of the remaining nodes
            for i in range(len(index)-1):
                for j in range(i+1, len(index)):
                    if index[i] < index[j]:
                        index[j] = index[j] - 1

            # Mask the selected nodes in x_masked and adjust the edge indices and attributes
            for element in index[-self.imp_mask_index:]:
                x_masked = torch.cat((x_masked[:element], x_masked[element+1:]))

                # Adjust the node indices in edge_index for the masked node
                masked_node_index = element
                num_nodes_deleted = self.imp_mask_index
                node_mask = (edge_index[0] != masked_node_index) & (edge_index[1] != masked_node_index)
                edge_index = edge_index[:, node_mask]
                edge_attr = edge_attr[node_mask]

                # Adjust the remaining node indices in edge_index after the deleted nodes
                edge_index[0] = torch.where(edge_index[0] > masked_node_index, edge_index[0] - 1, edge_index[0])
                edge_index[1] = torch.where(edge_index[1] > masked_node_index, edge_index[1] - 1, edge_index[1])

            # Adjust the batch size and node index for the masked node
            batch_size = test_batch.batch.max().item() + 1
            num_nodes = test_batch.num_nodes - self.imp_mask_index
            test_batch.batch = test_batch.batch[torch.cat((index[:-self.imp_mask_index], torch.tensor([0] * self.imp_mask_index)))]

        # Forward pass with masked input features and adjusted graph
        x_masked2 = x_masked
        forward_kwargs = {
            'edge_weight': edge_attr,
            'batch': torch.zeros(300 - self.imp_mask_index).to(device).to(torch.int64)
        }
        out = self.model(x=x_masked, edge_index=edge_index, forward_kwargs=forward_kwargs)

        loss_function = CrossEntropyLoss(weight=self.class_weights).to(device)
        test_loss = loss_function(out, test_batch.y)

        pred = out.argmax(-1)
        test_label = test_batch.y
        
        accuracy = (pred == test_label).sum().item() / pred.shape[0]

        # Log the test accuracy
        self.log("test_acc", accuracy)

        # Return the predicted labels and ground truth labels
        return pred, test_label
    
    def test_epoch_end(self, outputs):
        """
        This function receives accumulated predicted and true labels from the test_step and uses them on the confusion matrix and classification report that are than printed.
        
        Args:
            outputs: Contains an array of predicted and an array of true lables

        """
        global pred_array, true_array
        true_array=[outputs[i][1].cpu().numpy() for i in range(len(outputs))]
        pred_array = [outputs[i][0].cpu().numpy() for i in range(len(outputs))]  
        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array))
        
        
def main():
    """
    Main function for training.

    Affected by:
        Config["graph"]["classif"]
        Config["model"]
    """

    Config["graph"]["classif"] = "graph"
    Config["graph"]["type"] = "VG"
    Config["model"]["SEED"] = 280

    global device
    # initiate callback functions, DVC, Seed and device
    early_stop = EarlyStopping(monitor='val_acc',patience=Config["model"]["patience"], strict=False,verbose=False, mode='max')
    val_checkpoint_best_loss = ModelCheckpoint(filename="best_loss", monitor = "val_loss", mode="max")
    logger = DVCLiveLogger() # the bonus of using DVCLiveLogger() is that you can visualise the validation accuracy live in dvclive/report.html
    
    torch.manual_seed(Config["model"]["SEED"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # creates dataset containing graphs and the overall class_weights
    output, class_weights = Rutgers_graph() 

    # sets the train, validation and test sizes and atributes a number of time series coresponding to those sizes to every DataLoader
    train_size = int(Config["model"]["train/val/test"]["train"] * len(output))
    val_size = int(Config["model"]["train/val/test"]["val"]*len(output))
    test_size = len(output) - (val_size + train_size) 
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=Config["model"]["batch_size"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=int(Config["model"]["batch_size"]/2), shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


    if Config["model"]["status"] in ("train","full"):
        model = NGClassifier(class_weights=class_weights,model=GINE(),Config=Config).double()

        trainer = pl.Trainer(logger=logger,accelerator='gpu',devices=1)
        trainer.fit(model, train_loader, val_loader)

    if Config["model"]["status"] in ("test","full"): #gets the accuracy for anomaly type selected in Config["importance"]["anomaly_type"] and number of nodes deleted from graph in Config["importance"]["take_out"]

        model = NGClassifier.load_from_checkpoint(Config["importance"]["ckpt_path"],class_weights=class_weights,model=GINE(),Config=Config).double() #GINE is for graph classification
        importance_scores=create_node_importance_array(test_loader,model.model) # Importance scores for every graph in the testing loader

        test_loader.dataset.indices = importance_scores[Config["importance"]["anomaly_type"]][2]

        model.imp_mask_index = Config["importance"]["take_out"]
        model.imp_mask = importance_scores[Config["importance"]["anomaly_type"]][Config["importance"]["importance"]] # importance_scores[1][0] is for my importance matrix, while importance_scores[1][1] is the exp importance matrix from graphxai

        trainer = pl.Trainer(logger=False,accelerator='gpu',devices=1)

        trainer.test(model, test_loader)
    if Config["model"]["status"] in ("test_auto"): #gets the array where num_of_anom are anomaly types and num_of_imp is the number of steps it takes where it deletes the next 30 most important nodes  

        model = NGClassifier.load_from_checkpoint(Config["importance"]["ckpt_path"],class_weights=class_weights,model=GINE(),Config=Config).double() #GINE is for graph classification
        importance_scores=create_node_importance_array(test_loader,model.model)
        
        num_of_anom = 5
        num_of_imp = 9
        global imp_results
        imp_results = np.zeros((num_of_anom, num_of_imp))
        for i in range(num_of_anom):
            test_loader.dataset.indices = importance_scores[i][2]
            for j in range(num_of_imp):
                model.imp_mask_index = j*30
                model.imp_mask = importance_scores[i][Config["importance"]["importance"]] 

                trainer = pl.Trainer(logger=False,accelerator='gpu',devices=1)
                trainer.test(model, test_loader)
                imp_results[i][j] = accuracy_score(true_array, pred_array)
                print(accuracy_score(true_array, pred_array))

In [7]:
def report(csv_name,true_array, pred_array):
    """
    Generates a classification report based on the true and predicted arrays for a given utility.

    Args:
        csv_name: Name of the .csv file.
        true_array: Array of true labels.
        pred_array: Array of predicted labels.

    Returns:
        Saves the report as a CSV file.
    """
    print(csv_name)
    report = classification_report(true_array, pred_array, output_dict=True)
    df = pd.DataFrame(report).transpose()
    file_name = 'results_rutgers'+str(Config["model"]["SEED"])+'/'+csv_name
    df.to_csv(file_name +".csv")
    shutil.make_archive(file_name, 'zip', file_name)

# Explainer

In [8]:
import torch

from typing import Optional, Callable
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.data import Data

from graphxai.explainers._base import _BaseExplainer
from graphxai.utils import Explanation


class GradExplainer(_BaseExplainer):
    """
    Vanilla Gradient Explanation for GNNs

    Args:
        model (torch.nn.Module): model on which to make predictions
            The output of the model should be unnormalized class score.
            For example, last layer = CNConv or Linear.
        criterion (torch.nn.Module): loss function
    """
    def __init__(self, model: torch.nn.Module, 
            criterion: Callable[[torch.Tensor, torch.Tensor], torch.Tensor]):
        super().__init__(model)
        self.criterion = criterion

    def get_explanation_node(self, node_idx: int, x: torch.Tensor,
                             edge_index: torch.Tensor,
                             label: Optional[torch.Tensor] = None,
                             num_hops: Optional[int] = None,
                             aggregate_node_imp = torch.sum,
                             y = None,
                             forward_kwargs: dict = {}, **_) -> Explanation:
        """
        Explain a node prediction.

        Args:
            node_idx (int): index of the node to be explained
            x (torch.Tensor, [n x d]): node features
            edge_index (torch.Tensor, [2 x m]): edge index of the graph
            label (torch.Tensor, optional, [n x ...]): labels to explain
                If not provided, we use the output of the model.
                (:default: :obj:`None`)
            num_hops (int, optional): number of hops to consider
                If not provided, we use the number of graph layers of the GNN.
                (:default: :obj:`None`)
            aggregate_node_imp (function, optional): torch function that aggregates
                all node importance feature-wise scores across the enclosing 
                subgraph. Must support `dim` argument. 
                (:default: :obj:`torch.sum`)
            forward_kwargs (dict, optional): Additional arguments to model.forward 
                beyond x and edge_index. Must be keyed on argument name. 
                (default: :obj:`{}`)

        :rtype: :class:`graphxai.Explanation`

        Returns:
            exp (:class:`Explanation`): Explanation output from the method.
                Fields are:
                `feature_imp`: :obj:`torch.Tensor, [features,]`
                `node_imp`: :obj:`torch.Tensor, [nodes_in_khop, features]`
                `edge_imp`: :obj:`None`
                `enc_subgraph`: :obj:`graphxai.utils.EnclosingSubgraph`
        """
        label = self._predict(x, edge_index,
                              forward_kwargs=forward_kwargs) if label is None else label
        num_hops = self.L if num_hops is None else num_hops

        khop_info = subset, sub_edge_index, mapping, _ = \
            k_hop_subgraph(node_idx, num_hops, edge_index,
                           relabel_nodes=True, num_nodes=x.shape[0])
        sub_x = x[subset]

        self.model.eval()
        sub_x.requires_grad = True
        output = self.model(sub_x, sub_edge_index)
        loss = self.criterion(output[mapping], label[mapping])
        loss.backward()

        feature_imp = sub_x.grad[torch.where(subset == node_idx)].squeeze(0)
        node_imp = aggregate_node_imp(sub_x.grad, dim = 1)

        exp = Explanation(
            feature_imp = feature_imp, #[score_1, ]
            node_imp = node_imp, #[score_1, score_2, ...] [[], []] NxF
            node_idx = node_idx
        )

        exp.set_enclosing_subgraph(khop_info)

        return exp

    def get_explanation_graph(self, 
                                x: torch.Tensor, 
                                edge_index: torch.Tensor,
                                label: torch.Tensor, 
                                aggregate_node_imp = torch.sum,
                                forward_kwargs: dict = {}) -> Explanation:
        """
        Explain a whole-graph prediction.

        Args:
            x (torch.Tensor, [n x d]): node features
            edge_index (torch.Tensor, [2 x m]): edge index of the graph
            label (torch.Tensor, [n x ...]): labels to explain
            aggregate_node_imp (function, optional): torch function that aggregates
                all node importance feature-wise scores across the graph. 
                Must support `dim` argument. (:default: :obj:`torch.sum`)
            forward_kwargs (dict, optional): additional arguments to model.forward
                beyond x and edge_index

        :rtype: :class:`graphxai.Explanation`

        Returns:
            exp (:class:`Explanation`): Explanation output from the method. 
                Fields are:
                `feature_imp`: :obj:`None`
                `node_imp`: :obj:`torch.Tensor, [num_nodes, features]`
                `edge_imp`: :obj:`None`
                `graph`: :obj:`torch_geometric.data.Data`
        """

        self.model.eval()
        x.requires_grad = True
        output = self.model(x, edge_index, **forward_kwargs)[0]
        loss = self.criterion(output, label)
        loss.backward()

        node_imp = aggregate_node_imp(x.grad, dim = 1)

        exp = Explanation(
            node_imp = node_imp
        )

        exp.set_whole_graph(Data(x=x, edge_index=edge_index))

        return exp

    def get_explanation_link(self):
        """
        Explain a link prediction.
        """
        raise NotImplementedError()

In [9]:


def get_important_nodes(model, data):
    """
    calculates important nodes through gradient and gets the explanation from the GNNExplainer.
    
    Args:
        model: trained model
        data: test data that will be run on the model
        
    returns:
        node_importance: node importance scores calculated through gradient 
        exp: a class created by GraphXAI that contains important node scores that also works around gradient
        
    """
    
    # Enable gradient calculation
    data.x.requires_grad = True
    
    forward_kwargs={
        'edge_weight': data.edge_attr,#.to(device),
        'batch' : torch.tensor([0])#.to(device)
    }
    # Forward pass
    # print(data.x.device,data.edge_index.device,forward_kwargs['edge_weight'].device,forward_kwargs['batch'].device,model.device)
    out = model(x=data.x,edge_index=data.edge_index,forward_kwargs=forward_kwargs)
    # Calculate the loss (e.g., cross-entropy) based on the output and ground truth
    loss = F.cross_entropy(out[0], torch.tensor(data.y))

    # Compute gradients of the loss with respect to node embeddings
    loss.backward()

    # Calculate the node importance scores based on the gradients
    node_importance = torch.abs(data.x.grad).sum(dim=1)
    # exp=1
    data.x.requires_grad = False
    
    fwargs={
        'edge_weight': data.edge_attr,#.to(device),
        'batch' : torch.tensor([0])#.to(device)
    }

    EXP = GradExplainer(model,torch.nn.CrossEntropyLoss())
    exp = EXP.get_explanation_graph(
        x=data.x,#.to(device)
        edge_index=data.edge_index,#.to(device)
        label = data.y,#.to(device)
        forward_kwargs={'forward_kwargs':fwargs}
    )

    
    return node_importance, exp

In [41]:
def create_node_importance_array(test_loader,model):
    """
    Creates an array that contains 7 parameters:
        - node importance calculated by Gradient tehnique in function "get_important_nodes" 
        - node importance calculated by GraphXAI Gradient tehnique in function "get_important_nodes" 
        - indece of the graph whoes node importance was calculated 
        - node importance std
        - node importance med
        - node importance max
        - node importance min
    returns:
        importance_score: array containing all 7 parameters for all graphs inside the test loader
        
    """
    import pickle
    importance_score = [[[],[],[],[],[],[],[]],
            [[],[],[],[],[],[],[]],
            [[],[],[],[],[],[],[]],
            [[],[],[],[],[],[],[]],
            [[],[],[],[],[],[],[]]]
    for i, data in enumerate(test_loader.dataset):
        node_importance, exp = get_important_nodes(model, data)
        std_data = np.std(np.array(node_importance.cpu()))
        med_data = np.median(np.array(node_importance.cpu()))
        max_data = np.max(np.array(node_importance.cpu()))
        min_data = np.min(np.array(node_importance.cpu()))

        importance_score[data.y][0].append(node_importance)
        importance_score[data.y][1].append(exp.node_imp)
        importance_score[data.y][2].append(test_loader.dataset.indices[i])
        importance_score[data.y][3].append(std_data)
        importance_score[data.y][4].append(med_data)
        importance_score[data.y][5].append(max_data)
        importance_score[data.y][6].append(min_data)
        
        
        
    return importance_score

In [ ]:
def plot_importance_impact()
    """
    plots how the accuracy changes if we mask the important nodes by step of 30 nodes in the graph.
    Can be run if we insert Config["model"]["status"] = "test_auto" where the array of importance impact is given at the end
    """
    import matplotlib.pyplot as plt
    legend_labels = ["no_anom", "non_recovery", "recovery", "instant", "slow"]
    colors =["red","blue","yellow","green", "gray"]

    for i in range(5):
        plt.plot(imp_results[i], label=legend_labels[i], color = colors[i])

    plt.legend(labels=legend_labels)
    plt.savefig('reverse_importance_GraphXAI.png')
    plt.show()

# Main

In [19]:
"""
This list contains configurations for generating and training a graph from a Rutgers time series
"""
Config = {
    "graph": {
        "data_path" : "datasets/Rutgers/", # path to datasets
        "folder_name" : "dataset_uncut.csv", #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
        "properties_name" : "dataset_properties.npz",  # path to properties used for random 
        "mask_name" : "dataset_mask.npz", # path to mask dataset used for random
        "classif": "node",  # Type of classification (node or graph)
        "len_type" : "un/cut", #"un/cut", "random", the shape of data used in later paths 
        "type": "VG",  # Type of graph (MTF, VG, Dual_VG)
        "MTF": {
            "num_bins": "auto"  # Number of bins for MTF graph (integer or "auto")
        },
        "VG": {
            "edge_type": "natural",  # Type of edge calculation for VG graph (natural or horizontal)
            "distance": 'distance',  # Type of distance metric for VG graph
                                    # (slope, abs_slope, distance, h_distance, v_distance, abs_v_distance)
            "edge_dir": "directed"  # Directionality of edges in VG graph (undirected or directed)
        }
    },
    
    "model": {
        "SEED": 280,  # Random seed for reproducibility
        "learning_rate": 0.005,  # Learning rate for training
        "batch_size": 64,  # Batch size for training
        "range_epoch": 200,  # Number of training epochs
        "save_file": "test_test",  # File name for saving trained model
        "name_of_save": "test_u-time",  # Name of the save (e.g., checkpoint name)
        "patience": 500,  # Patience for early stopping
        "train/val/test": {
            "train": 0.8,  # Percentage of data used for training
            "val": 0.04,  # Percentage of data used for validation
            "test": 0.16  # Percentage of data used for testing
        },
        "status": "test_auto" #, "test", "full","test_auto" specifies what to do with the model
    },
    "importance":{
        "take_out": 0, # number of nodes taken from the graph, which is then sent through the model to make a prediction
        "ckpt_path": "Trained_models_Rutgers/VG_dir_seed=6000/version_3.ckpt", # path to the saved models parameters
        "anomaly_type": 1,
        "importance": 0 #0:refers to my importance matrix, while 1 refers to GraphXAI importance matrix
    }
}

In [ ]:
"""
This for loop runs the main function for graph generation, training and saving the testing results into a csv file.
The trained model can be found in DvcLiveLogger/dvclive_run/checkpoints.
"""
#Most common Config parameters
Config["graph"]["classif"] = "graph"
Config["graph"]["type"] = "VG"
Config["model"]["SEED"] = 280
Config["importance"]["anomaly_type"] = 1
Config["importance"]["importance"] = 0
Config["importance"]["take_out"] = 30

#For generating, training and testing the graph
main()

# Generate a classification report for the utility
# report(get_versions_TSSB()[utility][:-4],true_array, pred_array,2)

# plot_importance_impact()